In [1]:
import json
import pandas as pd

In [2]:
saoke = []
for line in open('SAOKE_DATA.json', 'r'):
    saoke.append(json.loads(line))

In [18]:
saoke[0]

{'logic': [{'object': ['[低山|山麓地带]的[次生阔叶林|阔叶林|针阔叶混交林]中'],
   'place': '_',
   'predicate': '主要栖息于',
   'qualifier': '_',
   'subject': '_',
   'time': '_'},
  {'object': ['[人工林|针叶林]'],
   'place': '_',
   'predicate': '也出入于',
   'qualifier': '_',
   'subject': '_',
   'time': '_'},
  {'object': ['海拔1700米的[中|高]山地带'],
   'place': '在北方',
   'predicate': '有时可上到',
   'qualifier': '_',
   'subject': '_',
   'time': '夏季'},
  {'object': ['海拔3000米左右的森林中'],
   'place': '在南方',
   'predicate': '甚至上到',
   'qualifier': '_',
   'subject': '_',
   'time': '夏季'},
  {'object': ['[山麓|邻近平原地带]的[次生阔叶林|人工林|林缘疏林灌丛]'],
   'place': '_',
   'predicate': '多下到',
   'qualifier': '_',
   'subject': '_',
   'time': '冬季'},
  {'object': ['[果园|道旁]|[地边树丛|[房前屋后|庭院中]的树上]'],
   'place': '_',
   'predicate': '有时也进到',
   'qualifier': '_',
   'subject': '_',
   'time': '_'}],
 'natural': '主要栖息于低山和山麓地带的次生阔叶林、阔叶林和针阔叶混交林中，也出入于人工林和针叶林，夏季在北方有时可上到海拔1700米的中、高山地带，在南方夏季甚至上到海拔3000米左右的森林中，冬季多下到山麓和邻近平原地带的次生阔叶林、人工林和林缘疏林灌丛，有时也进到果园、道旁和地边树丛、房前

In [63]:
sent = ''
for o in saoke[0]['logic']:
    sent += str(tuple([replaceMissinfo(o['subject'])]+ [replaceMisspred(o['predicate'])] + [replaceMissinfo(o['object'][0])]))
    sent += '$'

In [67]:
sent[:]

"('X', '主要栖息于', '[低山|山麓地带]的[次生阔叶林|阔叶林|针阔叶混交林]中')$('X', '也出入于', '[人工林|针叶林]')$('X', '有时可上到', '海拔1700米的[中|高]山地带')$('X', '甚至上到', '海拔3000米左右的森林中')$('X', '多下到', '[山麓|邻近平原地带]的[次生阔叶林|人工林|林缘疏林灌丛]')$('X', '有时也进到', '[果园|道旁]|[地边树丛|[房前屋后|庭院中]的树上]')$"

In [56]:
def replaceMissinfo(aaa):
    placeholder = ['Z','Y','X']
    for i in range(len(aaa)):
        if aaa[i] == '_':
            aaa = aaa[:i] + placeholder.pop() + aaa[i+1:]
    return aaa
def replaceMisspred(aaa):
    if aaa == '_':
        return 'P'
    else:
        return aaa
        

In [68]:
labels = []
for sentence in saoke:
    sent = ''
    for o in sentence['logic']:
        sent += str(tuple([replaceMissinfo(o['subject'])]+ [replaceMisspred(o['predicate'])] + [replaceMissinfo(o['object'][0])]))
        sent += '$'
    labels.append(sent[:-1])